# Super Store Analysis

## Installing dependencies

In [ ]:
# !pip install xlrd

## Import modules

In [1]:
import re
import warnings
import pandas as pd
import numpy as np

warnings.filterwarnings('ignore')

## Load data

In [ ]:
file_path = 'Sample - Superstore.xls'
orders_df = pd.read_excel(file_path, sheet_name=0, header=0)
people_df = pd.read_excel(file_path, sheet_name=1, header=0)
returns_df = pd.read_excel(file_path, sheet_name=2, header=0)

## Data exploration

### **Orders**

#### Reformat headers

In [ ]:
def snake_case(data:str) -> str:
	data = data.lower()
	return(data.strip().replace(' ','_'))

orders_df.columns = [snake_case(col) for col in orders_df.columns]

#### Overview

In [19]:
orders_df.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country/region,city,...,postal_code,region,product_id,category,sub-category,product_name,sales,quantity,discount,profit
0,1,US-2021-103800,2021-01-03,2021-01-07,Standard Class,DP-13000,Darren Powers,Consumer,United States,Houston,...,77095,Central,OFF-PA-10000174,Office Supplies,Paper,"Message Book, Wirebound, Four 5 1/2"" X 4"" Form...",16.448,2,0.2,5.5512
1,2,US-2021-112326,2021-01-04,2021-01-08,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,...,60540,Central,OFF-BI-10004094,Office Supplies,Binders,GBC Standard Plastic Binding Systems Combs,3.540,2,0.8,-5.4870
2,3,US-2021-112326,2021-01-04,2021-01-08,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,...,60540,Central,OFF-LA-10003223,Office Supplies,Labels,Avery 508,11.784,3,0.2,4.2717
3,4,US-2021-112326,2021-01-04,2021-01-08,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,...,60540,Central,OFF-ST-10002743,Office Supplies,Storage,SAFCO Boltless Steel Shelving,272.736,3,0.2,-64.7748
4,5,US-2021-141817,2021-01-05,2021-01-12,Standard Class,MB-18085,Mick Brown,Consumer,United States,Philadelphia,...,19143,East,OFF-AR-10003478,Office Supplies,Art,Avery Hi-Liter EverBold Pen Style Fluorescent ...,19.536,3,0.2,4.8840


In [20]:
orders_df.tail()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country/region,city,...,postal_code,region,product_id,category,sub-category,product_name,sales,quantity,discount,profit
10189,10190,US-2024-143259,2024-12-30,2025-01-03,Standard Class,PO-18865,Patrick O'Donnell,Consumer,United States,New York City,...,10009,East,OFF-BI-10003684,Office Supplies,Binders,Wilson Jones Legal Size Ring Binders,52.776,3,0.2,19.7910
10190,10191,US-2024-115427,2024-12-30,2025-01-03,Standard Class,EB-13975,Erica Bern,Corporate,United States,Fairfield,...,94533,West,OFF-BI-10004632,Office Supplies,Binders,GBC Binding covers,20.720,2,0.2,6.4750
10191,10192,US-2024-156720,2024-12-30,2025-01-03,Standard Class,JM-15580,Jill Matthias,Consumer,United States,Loveland,...,80538,West,OFF-FA-10003472,Office Supplies,Fasteners,Bagged Rubber Bands,3.024,3,0.2,-0.6048
10192,10193,US-2024-143259,2024-12-30,2025-01-03,Standard Class,PO-18865,Patrick O'Donnell,Consumer,United States,New York City,...,10009,East,TEC-PH-10004774,Technology,Phones,Gear Head AU3700S Headset,90.930,7,0.0,2.7279
10193,10194,CA-2024-143500,2024-12-30,2025-01-03,Standard Class,HO-15230,Harry Olson,Consumer,Canada,Charlottetown,...,C0A,East,OFF-BI-10004040,Office Supplies,Binders,Wilson Jones Impact Binders,3.024,3,0.2,-0.6048


#### Details

- measurement in rows x columns
- columns and datatypes

In [25]:
print('(row, col)')
orders_df.shape

(row, col)


(10194, 21)

In [ ]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10194 entries, 0 to 10193
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   row_id          10194 non-null  int64         
 1   order_id        10194 non-null  object        
 2   order_date      10194 non-null  datetime64[ns]
 3   ship_date       10194 non-null  datetime64[ns]
 4   ship_mode       10194 non-null  object        
 5   customer_id     10194 non-null  object        
 6   customer_name   10194 non-null  object        
 7   segment         10194 non-null  object        
 8   country/region  10194 non-null  object        
 9   city            10194 non-null  object        
 10  state/province  10194 non-null  object        
 11  postal_code     10194 non-null  object        
 12  region          10194 non-null  object        
 13  product_id      10194 non-null  object        
 14  category        10194 non-null  object        
 15  su

(10194, 21)

### **People**
